In [4]:
import cv2
import os
import time
import numpy as np
import threading
import yagmail
from keras_facenet import FaceNet
import pickle

# Initialize OpenCV face detection using Haar Cascade
haar_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# Load FaceNet model
face_net = FaceNet()

# Load face database
with open("data.pkl", "rb") as file:
    database = pickle.load(file)

# Email configuration
SMTP_USERNAME = 'your.gmail@gmail.com'  # Update with your Gmail email address
SMTP_APP_PASSWORD = 'your_app_password'  # Update with your Gmail app password
RECIPIENT_EMAIL = 'recipient@example.com'
EMAIL_SUBJECT = 'Unknown Face Detected at your house'

# Directory to save unknown face images
unknown_faces_dir = "unknown_faces"
os.makedirs(unknown_faces_dir, exist_ok=True)

# Initialize yagmail instance
yag = yagmail.SMTP(SMTP_USERNAME, SMTP_PASSWORD)

# Function to send email notification
def send_email_notification(filename):
    body = "Unknown face detected. Please check the attached image."
    
    try:
        yag.send(
            to=RECIPIENT_EMAIL,
            subject=EMAIL_SUBJECT,
            contents=body,
            attachments=filename
        )
        print("Email notification sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")

# Function to process each frame
def process_frame(frame):
    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    
    for (x, y, w, h) in faces:
        face_img = cv2.resize(frame[y:y+h, x:x+w], (160, 160))
        face_signature = face_net.embeddings(np.expand_dims(face_img, axis=0))
        
        min_dist = 0.94
        identity = 'Unknown'
        
        # Check distance to known faces in the database
        for key, value in database.items():
            dist = np.linalg.norm(value - face_signature)
            if dist < min_dist:
                min_dist = dist
                identity = key
        
        # Draw rectangle around the face
        color = (0, 0, 255) if identity == 'Unknown' else (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, identity, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        # Save frame if unknown face detected
        if identity == 'Unknown':
            unknown_face_filename = os.path.join(unknown_faces_dir, f"unknown_face_{time.strftime('%Y%m%d%H%M%S')}.jpg")
            cv2.imwrite(unknown_face_filename, frame)
            print("Unknown face captured and saved.")
            
            # Send email notification in a separate thread
            email_thread = threading.Thread(target=send_email_notification, args=(unknown_face_filename,))
            email_thread.start()

# Main function
def main():
    cap = cv2.VideoCapture(0)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                continue
            
            current_datetime = time.strftime("%Y-%m-%d %H:%M:%S")
            cv2.putText(frame, current_datetime, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            
            # Process every alternate frame to reduce load
            process_frame(frame)
            
            # Display the resulting frame
            cv2.imshow('frame', frame)
            
            # Break the loop on 'ESC' key press
            if cv2.waitKey(1) & 0xFF == 27:
                break

    finally:
        # Release video capture and close windows
        cap.release()
        cv2.destroyAllWindows()

        # Close yagmail connection
        yag.close()

if __name__ == "__main__":
    main()


1/1 [==============================] - 1s 942ms/step
Unknown face captured and saved.
1/1 [==============================] - 0s 42ms/step
Unknown face captured and saved.
1/1 [==============================] - 0s 46ms/step
Unknown face captured and saved.
1/1 [==============================] - 0s 42ms/step
Unknown face captured and saved.
Error sending email: (503, b'5.5.1 RCPT first. A mail transaction protocol command was issued out of\n5.5.1 sequence. For more information, go to\n5.5.1  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.5.1 specifications. h10-20020a170902f54a00b001e61d87e4aasm2872024plf.185 - gsmtp')
1/1 [==============================] - 0s 43ms/step
Unknown face captured and saved.
Email notification sent successfully.
Error sending email: (250, b'2.0.0 OK  1713517732 s1-20020a170902ea0100b001e3d2314f3csm2866742plg.141 - gsmtp')
1/1 [==============================] - 0s 60ms/step
Unknown face captured and saved.
1/1 [==============================

Error sending email: (451, b'4.4.2 Timeout - closing connection.\n4.4.2  For more information, go to\n4.4.2  https://support.google.com/a/answer/3221692 s1-20020a170902ea0100b001e3d2314f3csm2866742plg.141 - gsmtp')
